In [18]:
import pandas as pd
import numpy as np
from collections import OrderedDict
from scipy import constants as const

In [31]:
switcheskeys   = ['RFswitch','betatronSwitch','raddampSwitch','IBSswitch','collimationSwitch','blowupSwitch','collisionSwitch','writeAllCoordSwitch', 'writeMountSwitch']
ringparamskeys = ['aatomb2', 'qatomb2', 'aatomb2', 'qatomb2', 'proton', 'ion', 'thib', 'nturns', 'nMacro', 'timeRatio', 'nwrite', 'gammat', 'circ', 'gamma1', 
                  'gamma2', 'vrfb2', 'nharmb2', 'vrf2b2', 'nharm2b2', 'vrfb2', 'nharmb2', 'vrf2b2', 'nharm2b2', 'tunex1', 'tuney1', 'chromx1', 'chromy1', 
                  'tunex2', 'tuney2', 'chromx2', 'chromy2', 'dqmin1', 'dqmin2', 'k2L1', 'k2Lskew2', 'k2L2', 'k2Lskew1', 'tfsb2', 'tfsb2']
startingconditionskeys = ['tauhat1', 'tauhat2', 'longcoor', 'transcoor', 'bunchLenPrecis', 'power', 'alint']
radiationdampingkeys   = ['radMethod', 'tradlong', 'tradperp', 'siglong', 'sigperp', 'rho0']
ibsdictkeys            = ['ibsMethod', 'coupleIBS', 'coulombLog', 'fracibstot', 'nbins', 'piwinski', 'bane', 'intfile']
colldictkeys           = ['refEmxy', 'cutoffAmpl', 'collimAvgSwitch', 'emitMethod', 'nSigCutBeta', 'nSigCutMom', 'betaxMom', 'dispxMom']
blowupdictkeys         = ['pxKickFac', 'pyKickFac', 'blowupMethod']
collisiondictkeys      = ['collRoutine', 'sigI', 'longIntBins', 'ips_leveling', 'ips_leveling_values']

# Process and writing switches 

In [19]:
switches                        = OrderedDict()
switches['RFswitch']            = True    # RFswitch (set to 1 to activate synchrotron motion)
switches['betatronSwitch']      = True    # betatronSwitch (set to 1 to activate betatron motion)
switches['raddampSwitch']       = True    # raddampSwitch (set to 1 to activate radiation damping and quantum excitation)
switches['IBSswitch']           = True    # IBSswitch (set to 1 to activate ibs)
switches['collimationSwitch']   = True    # collimationSwitch (set to 1 to activate losses on aperture cuts)
switches['blowupSwitch']        = False    # blowupSwitch (set to 1 to activate artificial blowup - ADT loss maps etc)
switches['collisionSwitch']     = True    # collisionSwitch (set to 1 to activate collisions)
switches['writeAllCoordSwitch'] = False    # writeAllCoordSwitch
switches['writeMountSwitch']    = False    # writeMountSwitch

# Ring parameters 

In [20]:
# constants
protonmass                      = const.physical_constants['proton mass energy equivalent in MeV'][0]/1000 # GeV
electroncharge                  = const.physical_constants['elementary charge'][0]

ringparams                      = OrderedDict()
ringparams['aatomb1']           = 208         # atomic number of particle in beam 1
ringparams['aatomb2']           = 208         # atomic number of particle in beam 2

ringparams['nturns']            = 500       # number of turns to simulate in the machine
ringparams['nMacro']            = 1000      # number of simulated particles
ringparams['timeRatio']         = 20000       # (=real turns/sim.turns)
ringparams['nwrite']            = 100         # step interval in turns to calculate output variables and write them to files

ringparams['vrfb1']             = 0.0
ringparams['nharmb1']           = 360
ringparams['vrf2b1']            = -12.0e6
ringparams['nharm2b1']          = 35640

ringparams['vrfb2']             = 0.0
ringparams['nharmb2']           = 360
ringparams['vrf2b2']            = -12.0e6
ringparams['nharm2b2']          = 35640

# set general parameters
ringparams['proton']            = protonmass
ringparams['ion']               = 193.7291748489224
ringparams['thib']              = 2.5e-9      # RF period
ringparams['dqmin1']            = 0.0     # (coupling between x-py, y-px)
ringparams['dqmin2']            = 0.0      # (coupling between x-py, y-px)
ringparams['k2L1']              = 1.1       # (thin sextupole strengths)
ringparams['k2Lskew1']          = 0.0        # (thin sextupole strengths)
ringparams['k2L2']              = 1.1       # (thin sextupole strengths)
ringparams['k2Lskew2']          = 0.0        # (thin sextupole strengths)


# Some of the parameters are determined directly from tfs files
# so it is very important to set the correct ones for BOTH beams !!!
ringparams['tfsb1']             = 'lhcb1-PbPb-6500.tfs'
ringparams['tfsb2']             = 'lhcb2-PbPb-6500.tfs'
# getting some of the settings from the tfs files
dftfsb1head                     = pd.read_csv(ringparams['tfsb1'],nrows=44,delim_whitespace=True)
ringparams['tunex1']            = float(dftfsb1head[dftfsb1head['NAME']=='Q1']['TWISS'].values[0])
ringparams['tuney1']            = float(dftfsb1head[dftfsb1head['NAME']=='Q2']['TWISS'].values[0])
ringparams['chromx1']           = float(dftfsb1head[dftfsb1head['NAME']=='DQ1']['TWISS'].values[0])
ringparams['chromy1']           = float(dftfsb1head[dftfsb1head['NAME']=='DQ2']['TWISS'].values[0])

dftfsb2head                     = pd.read_csv(ringparams['tfsb2'],nrows=44,delim_whitespace=True)
ringparams['tunex2']            = float(dftfsb2head[dftfsb2head['NAME']=='Q1']['TWISS'].values[0])
ringparams['tuney2']            = float(dftfsb2head[dftfsb2head['NAME']=='Q2']['TWISS'].values[0])
ringparams['chromx2']           = float(dftfsb2head[dftfsb2head['NAME']=='DQ1']['TWISS'].values[0])
ringparams['chromy2']           = float(dftfsb2head[dftfsb2head['NAME']=='DQ2']['TWISS'].values[0])

energy                          = float(dftfsb1head[dftfsb1head['NAME']=='ENERGY']['TWISS'].values[0])/float(dftfsb1head[dftfsb1head['NAME']=='CHARGE']['TWISS'].values[0])
ringparams['gammat']            = float(dftfsb1head[dftfsb1head['NAME']=='GAMMATR']['TWISS'].values[0])
ringparams['circ']              = float(dftfsb1head[dftfsb1head['NAME']=='LENGTH']['TWISS'].values[0])
ringparams['qatomb1']           = float(dftfsb1head[dftfsb1head['NAME']=='CHARGE']['TWISS'].values[0])
ringparams['qatomb2']           = float(dftfsb2head[dftfsb2head['NAME']=='CHARGE']['TWISS'].values[0])
ringparams['gamma1']            = float(dftfsb1head[dftfsb1head['NAME']=='GAMMA']['TWISS'].values[0])
ringparams['gamma2']            = float(dftfsb2head[dftfsb2head['NAME']=='GAMMA']['TWISS'].values[0])

# Radiation damping parameters 

In [21]:
radiationdamping                     = OrderedDict()
radiationdamping['radMethod']        = 'lattic'          # adMethod: can be manual (input next line), approx (smooth latt. I4=0), or lattic (rad. int., twiss file required)
radiationdamping['tradlong']         = 23519.0           # in sec
radiationdamping['tradperp']         = 47072.0           # in sec
radiationdamping['siglong']          = 4.64e-12          # (eq. sigma from raddamp-exit.)
radiationdamping['sigperp']          = 7.43e-8           # (m) (only used with manual method)
radiationdamping['rho0']             = 2784.32           # (dipole bend. radius in m, used only with approx)

# IBS settings 

In [24]:
ibsdict                              = OrderedDict()
ibsdict['ibsMethod']                 = 'nagaitsev'       # can be nagaitsev, piwiSmooth, piwLattice, modPiwLatt, baneApprox or interpolat
ibsdict['coupleIBS']                 = False                 # coupleIBS (0 gives separate growth rates, 1 gives same growth in x and y)
ibsdict['coulombLog']                = 20.0              # (used in nagaitsev method)
ibsdict['fracibstot']                = 1                 # factor of ibs strength
ibsdict['nbins']                     = 500               # number of bins
ibsdict['piwinski']                  = '/home/roderik/My_CERN_work/piwinski-ibs/ibs-rates-LHC-collision.dat'            
ibsdict['bane']                      = '/home/roderik/My_CERN_work/bane-ibs/gBaneTab.dat'              
ibsdict['intfile']                   = 'interpolate.dat'

# Collimation settings 

In [26]:
colldict                     = OrderedDict()
colldict['refEmxy']          = 5.06158E-10
colldict['cutoffAmpl']       = 12.0 #5.7           # (ref. sigma at which init. distr. is cut)
colldict['collimAvgSwitch']  = 0            # 
colldict['emitMethod']       = 'stdev'       # (stdev,stcal,exfit)
colldict['nSigCutBeta']      = 12.0#5.7           # (ref. sigma at which init. distr. is cut)
colldict['nSigCutMom']       = 10.0          # (momentum cut in beta-sigma)
colldict['betaxMom']         = 131.7        
colldict['dispxMom']         = 2.15         

#  Blowup

In [28]:
blowupdict                 = OrderedDict()
blowupdict['pxKickFac']    = 0.02 
blowupdict['pyKickFac']    = 0.0
blowupdict['blowupMethod'] = 'unSum'         # (unifo,gauss,unSum)

# Collisions

In [30]:
collisiondict                      = OrderedDict()
collisiondict['collRoutine']       = '6a'           #(1d is slow but without assumptions on distributions, 6a is fast with assumed Gaussian transverse)
collisiondict['sigI']              = 2.0          # (cross section for particle removal in collisions) 
collisiondict['longIntBins']       = 100
collisiondict['ips_leveling']      = {'IP1':False,'IP2':True,'IP5':False,'IP8':False}# leveling at IP1,IP2,IP5,IP8
collisiondict['ips_leveling_values'] = {'IP2':2.27e24}

# Some additional starting parameters

In [33]:
startingconditions                   = OrderedDict()
startingconditions['tauhat1']        = 1.25e-9  # half of bucket length
startingconditions['tauhat2']        = 1.25e-9
startingconditions['longcoor']       = 3                           # (0:parabolic with smokering, 1: read from file, 2: bi-Gaussian, 3: pseudo-Gaussian,exactly matched)
startingconditions['transcoor']      = 2                           # transvCoordMethod (1 or 2)
startingconditions['bunchLenPrecis'] = 0.01                        # (for longCoordMethod=3)
startingconditions['power']          = 0.75                        # (for longCoordMethod=0)
startingconditions['alint']          = 5                           # (for longCoordMethod=0)

# Create dataframe and write to csv for input  

In [37]:
inputdf = pd.DataFrame()
inputdict = switches.copy()
inputdict.update(ringparams)
inputdict.update(startingconditions)
inputdict.update(radiationdamping)
inputdict.update(ibsdict)
inputdict.update(colldict)
inputdict.update(blowupdict)
inputdict.update(collisiondict)
inputdict
rowtoadd = pd.DataFrame(inputdict,index=[0])
rowtoadd['ips_leveling'] = str(collisiondict['ips_leveling'])
rowtoadd['ips_leveling_values'] = str(collisiondict['ips_leveling_values'])
inputdf = inputdf.append(inputdict,ignore_index=True)

In [86]:
inputdf.to_csv('ctefsinput.csv',index=False)